# Project 2: Adversarial Search in AI 
## Applying Kalman Filters for Airplane Tracking 

# Import Statements

In [ ]:
from project2_base import *
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
import seaborn as sns
import traffic
from pykalman import KalmanFilter
from datetime import timedelta
from geopy import distance as dist
from geopy.distance import geodesic
from copy import deepcopy

# Data Loading and Initial Exploration

In [ ]:
# Load the data
datas = get_ground_truth_data()
print(datas)
print(type(datas))

# Extract a list of flight IDs and store it as a variable called flight_ids
flight_ids = list(datas.keys())
print(flight_ids)

# Extract a list of tuples with (flight_id, flight_data)
flights_info = [(flight_id, flight_data) for flight_id, flight_data in datas.items()]
print(flights_info)

### Analyse the first flight  

In [ ]:
datas["IGRAD_000"]
print (type(datas["IGRAD_000"]))

In [ ]:
dict(datas["IGRAD_000"])

In [ ]:
datas["IGRAD_000"].data

In [ ]:
print(type(datas["IGRAD_000"].data))

In [ ]:
datas["IGRAD_000"].between("2020-04-15 07:00", "2020-04-15 12:00")

In [ ]:
datas["IGRAD_000"].between("2020-04-15 07:00", "2020-04-15 12:00").data

# Data Analysis

### Helper functions to calculate duration of flight, altitude, and flight complexity


In [ ]:
def calculate_duration(flight_data):
    """
    Calculates the duration of the flight
    
    Args:
        flight_data (pandas.DataFrame): The flight data containing 'timestamp' column.
    
    Returns:
        pandas.Timedelta: The duration of the flight.
    """
    duration = flight_data['timestamp'].max() - flight_data['timestamp'].min()
    return duration

In [ ]:
def analyze_altitude(flight_data):
    """
    Analyzes the altitude data of the flight.
    
    Args:
        flight_data (pandas.DataFrame): The flight data containing 'altitude' column.
    
    Returns:
        dict: A dictionary containing max, min, and average altitude.
    """
    max_altitude = flight_data['altitude'].max()
    min_altitude = flight_data['altitude'].min()
    avg_altitude = flight_data['altitude'].mean()
    
    return {'max_altitude': max_altitude, 'min_altitude': min_altitude, 'avg_altitude': avg_altitude}

In [ ]:
def estimate_route_complexity(flight_data, change_threshold=20):
    """
    Estimates the route complexity by counting the number of significant directional changes.
    
    Args:
        flight_data (pandas.DataFrame): The flight data containing 'latitude' and 'longitude' columns.
        change_threshold (int): The change in bearing considered significant, in degrees.
    
    Returns:
        int: The estimated route complexity, defined by the number of significant turns.
    """
    num_turns = 0
    last_bearing = None
    
    for i in range(1, len(flight_data)):
        prev_point = (flight_data.iloc[i - 1]['latitude'], flight_data.iloc[i - 1]['longitude'])
        current_point = (flight_data.iloc[i]['latitude'], flight_data.iloc[i]['longitude'])
        
        # Calculate bearing between consecutive points
        delta_lon = np.radians(current_point[1] - prev_point[1])
        y = np.sin(delta_lon) * np.cos(np.radians(current_point[0]))
        x = np.cos(np.radians(prev_point[0])) * np.sin(np.radians(current_point[0])) - \
            np.sin(np.radians(prev_point[0])) * np.cos(np.radians(current_point[0])) * np.cos(delta_lon)
        current_bearing = np.degrees(np.arctan2(y, x)) % 360
        
        if last_bearing is not None:
            bearing_change = abs(current_bearing - last_bearing)
            bearing_change = min(bearing_change, 360 - bearing_change)  # Correct for angular wraparound
            if bearing_change > change_threshold:
                num_turns += 1
        
        last_bearing = current_bearing
    
    return num_turns

### Anaylse Duration, Altitude, and Route Complexity for each flight

In [ ]:
# Initialize a dictionary to store the results
results = {}

# Iterating through each flight in the dataset
for flight_id, flight in datas.items():
    
    # Access the DataFrame directly
    flight_data = flight.data  # Since it's already a DataFrame
    
    # Ensure the DataFrame is not empty
    if not flight_data.empty:
        # Calculate flight duration
        duration = calculate_duration(flight_data)

        # Analyze altitude data
        altitude_info = analyze_altitude(flight_data)

        # Estimate route complexity
        complexity = estimate_route_complexity(flight_data)  # Make sure this function is defined and adapted to your data structure

        # Store the results for the current flight
        results[flight_id] = {
            'duration': duration,
            'max_altitude': altitude_info['max_altitude'],
            'min_altitude': altitude_info['min_altitude'],
            'avg_altitude': altitude_info['avg_altitude'],
            'route_complexity': complexity
        }
    else:
        print(f"Warning: No data available for flight {flight_id}")

# Print out the results for each flight
for flight_id, metrics in results.items():
    print(f"Flight ID: {flight_id}")
    print(f"Duration: {metrics['duration']}")
    print(f"Max Altitude: {metrics['max_altitude']} meters")
    print(f"Min Altitude: {metrics['min_altitude']} meters")
    print(f"Average Altitude: {metrics['avg_altitude']} meters")
    print(f"Route Complexity (number of significant turns): {metrics['route_complexity']}")
    print("---------------------------------------------------------")

### Plot the graph

In [ ]:
# Prepare lists for plotting
flight_ids = list(results.keys())
durations = [result['duration'].total_seconds()/3600 for result in results.values()]  # Convert duration to hours
max_altitudes = [result['max_altitude'] for result in results.values()]
min_altitudes = [result['min_altitude'] for result in results.values()]
avg_altitudes = [result['avg_altitude'] for result in results.values()]
route_complexities = [result['route_complexity'] for result in results.values()]

# Plotting
plt.figure(figsize=(14, 8))

# Plot for flight duration
plt.subplot(2, 2, 1)  # 2 rows, 2 columns, 1st subplot
plt.bar(flight_ids, durations, color='skyblue')
plt.title('Flight Duration (Hours)')
plt.xticks(rotation=90)

# Plot for maximum altitude
plt.subplot(2, 2, 2)  # 2 rows, 2 columns, 2nd subplot
plt.bar(flight_ids, max_altitudes, color='lightgreen')
plt.title('Maximum Altitude (Meters)')
plt.xticks(rotation=90)

# Plot for average altitude
plt.subplot(2, 2, 3)  # 2 rows, 2 columns, 3rd subplot
plt.bar(flight_ids, avg_altitudes, color='salmon')
plt.title('Average Altitude (Meters)')
plt.xticks(rotation=90)

# Plot for route complexity
plt.subplot(2, 2, 4)  # 2 rows, 2 columns, 4th subplot
plt.bar(flight_ids, route_complexities, color='gold')
plt.title('Route Complexity (Number of Significant Turns)')
plt.xticks(rotation=90)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()

# Plot Functions

In [ ]:
def plot_actual_radar_all(actual_data, radar_data, 
                          n_minor_ticks_x=0, n_minor_ticks_longitude=0, 
                          n_minor_ticks_latitude=0, 
                          actual_data_label_latitude='Latitude (Actual)', radar_data_label_latitude='Latitude (Radar)',
                          actual_data_label_longitude='Longitude (Actual)', radar_data_label_longitude='Longitude (Radar)'):
    """Plots a graph of longitude, latitude against time for both actual and radar flight data.

    Args:
        actual_data (panda.DataFrame): The actual data to be plotted
        radar_data (panda.DataFrame): The radar data to be plotted
        n_minor_ticks_x (integer): The number of minor axis intervals for time axis
        n_minor_ticks_longitude (integer): The number of minor axis intervals for longitude axis
        n_minor_ticks_latitude (integer): The number of minor axis intervals for latitude axis
        actual_data_label_latitude (string): Label for actual data (Latitude) 
        radar_data_label_latitude (string): Label for radar data (Latitude)
        actual_data_label_longitude (string): Label for actual data (Longitude) 
        radar_data_label_longitude (string): Label for radar data (Longitude) 
    """
    # Plotting
    fig, ax1 = plt.subplots()

    # Set the x-axis as the timestamp
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Latitude', color='tab:red')
    ax1.plot(actual_data['timestamp'], actual_data['latitude'], color='tab:red', label=actual_data_label_latitude)
    ax1.plot(radar_data['timestamp'], radar_data['latitude'], color='tab:pink', label=radar_data_label_latitude, linestyle='--')
    ax1.tick_params(axis='y', labelcolor='tab:red')

    # Create a second y-axis for longitude
    ax2 = ax1.twinx()
    ax2.set_ylabel('Longitude', color='tab:blue')
    ax2.plot(actual_data['timestamp'], actual_data['longitude'], color='tab:blue', label=actual_data_label_longitude)
    ax2.plot(radar_data['timestamp'], radar_data['longitude'], color='tab:cyan', label=radar_data_label_longitude, linestyle='--')
    ax2.tick_params(axis='y', labelcolor='tab:blue')

    # Add a title
    plt.title('Latitude and Longitude over Time')

    # Show legends
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    # Set minor ticks between the major ticks
    ax1.xaxis.set_minor_locator(AutoMinorLocator(n_minor_ticks_x))
    ax1.yaxis.set_minor_locator(AutoMinorLocator(n_minor_ticks_latitude))
    ax2.yaxis.set_minor_locator(AutoMinorLocator(n_minor_ticks_longitude)) 

    # Color of the minor ticks
    ax1.tick_params(axis='y', which='minor', color='tab:red')
    ax2.tick_params(axis='y', which='minor', color='tab:blue')

    # Add vertical grid lines to the plot
    ax1.grid(True, which='both', axis='x', linestyle='--', linewidth=0.5)

    # Add horizontal grid lines to the plot for latitude
    ax1.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, color='tab:red', alpha=0.5)

    # Add horizontal grid lines to the plot for longitude
    ax2.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, color='tab:blue', alpha=0.5)

    plt.show()

In [ ]:
def plot_actual_radar_latitude(actual_data, radar_data, 
                               n_minor_ticks_x=0, n_minor_ticks_latitude=0,
                               actual_data_label_latitude='Latitude (Actual)', radar_data_label_latitude='Latitude (Radar)'):
    """Plots a graph of latitude against time for both actual and radar flight data.

    Args:
        actual_data (panda.DataFrame): The actual data to be plotted
        radar_data (panda.DataFrame): The radar data to be plotted
        n_minor_ticks_x (integer): The number of minor axis intervals for time axis
        n_minor_ticks_latitude (integer): The number of minor axis intervals for latitude axis
        actual_data_label_latitude (string): Label for actual data (Latitude) 
        radar_data_label_latitude (string): Label for radar data (Latitude)
    """
    # Plotting
    fig, ax1 = plt.subplots()

    # Set the x-axis as the timestamp
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Latitude', color='tab:red')
    ax1.plot(actual_data['timestamp'], actual_data['latitude'], color='tab:red', label=actual_data_label_latitude)
    ax1.plot(radar_data['timestamp'], radar_data['latitude'], color='tab:pink', label=radar_data_label_latitude, linestyle='--')
    ax1.tick_params(axis='y', labelcolor='tab:red')
    
    # Add a title
    plt.title('Latitude over Time')

    # Show legends
    ax1.legend(loc='upper left')

    # Set minor ticks between the major ticks
    ax1.xaxis.set_minor_locator(AutoMinorLocator(n_minor_ticks_x))
    ax1.yaxis.set_minor_locator(AutoMinorLocator(n_minor_ticks_latitude))

    # Color of the minor ticks
    ax1.tick_params(axis='y', which='minor', color='tab:red')

    # Add vertical grid lines to the plot
    ax1.grid(True, which='both', axis='x', linestyle='--', linewidth=0.5)

    # Add horizontal grid lines to the plot for latitude
    ax1.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, color='tab:red', alpha=0.5)

    plt.show()

In [ ]:
def plot_actual_radar_longitude(actual_data, radar_data, 
                                n_minor_ticks_x=0, n_minor_ticks_longitude=0,
                                actual_data_label_longitude='Longitude (Actual)', radar_data_label_longitude='Longitude (Radar)'):
    """Plots a graph of longitude against time for both actual and radar flight data.

    Args:
        actual_data (panda.DataFrame): The actual data to be plotted
        radar_data (panda.DataFrame): The radar data to be plotted
        n_minor_ticks_x (integer): The number of minor axis intervals for time axis
        n_minor_ticks_longitude (integer): The number of minor axis intervals for longitude axis
        actual_data_label_longitude='Longitude (Actual)', radar_data_label_longitude='Longitude (Radar)'
        actual_data_label_longitude (string): Label for actual data (Longitude) 
        radar_data_label_longitude (string): Label for radar data (Longitude) 
    """
    # Plotting
    fig, ax1 = plt.subplots()

    # Set the x-axis as the timestamp
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Longitude', color='tab:red')
    ax1.plot(actual_data['timestamp'], actual_data['longitude'], color='tab:red', label=actual_data_label_longitude)
    ax1.plot(radar_data['timestamp'], radar_data['longitude'], color='tab:pink', label=radar_data_label_longitude, linestyle='--')
    ax1.tick_params(axis='y', labelcolor='tab:red')

    # Add a title
    plt.title('Longitude over Time')

    # Show legends
    ax1.legend(loc='upper left')

    # Set minor ticks between the major ticks
    ax1.xaxis.set_minor_locator(AutoMinorLocator(n_minor_ticks_x))
    ax1.yaxis.set_minor_locator(AutoMinorLocator(n_minor_ticks_longitude))

    # Color of the minor ticks
    ax1.tick_params(axis='y', which='minor', color='tab:blue')

    # Add vertical grid lines to the plot
    ax1.grid(True, which='both', axis='x', linestyle='--', linewidth=0.5)

    # Add horizontal grid lines to the plot for longitude
    ax1.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, color='tab:red', alpha=0.5)

    plt.show()

# Kalman Filtering

### Define Kalman Filter Parameters

In [ ]:
def construct_kalman_filter(radar_data, delta_time=10):
    """Takes the radar data and defines the kalman filter model for the radar model

    Args:
        radar_data (pandas.DataFrame): Radar data to be used for constructing the Kalman Filter model.
        delta_time (int, optional): The time between each sampling. Defaults to 10.
    """

    dim_x = 4
    dim_z = 2
    standard_deviation_process = 1.5  # Standard Deviation for acceleration  (Plane example)
    standard_deviation_observation = 3  # Standard Deviation for radar measurement

    # Initialize the Kalman Filter
    kf = KalmanFilter(dim_x, dim_z)

    # Define the initial state estimate
    initial_position = np.array([radar_data.x.iloc[0], radar_data.y.iloc[0], 0., 0.])  # Assuming initial velocity is 0
    kf.initial_state_mean = initial_position

    # Define the state transition matrix (F)
    kf.transition_matrices = np.array([[1, 0, delta_time, 0],
                    [0, 1, 0, delta_time],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])

    # Define the process noise covariance matrix (Q)

    a = (1/4 * delta_time**4) * (standard_deviation_process**2)
    b = (1/2 * delta_time**3) * (standard_deviation_process**2)
    c = delta_time**2 * (standard_deviation_process**2)
    kf.transition_covariance = np.array(
                    [[a, 0, b, 0],
                    [0, a, 0, b],
                    [b, 0, c, 0],
                    [0, b, 0, c]])

    # Define the observation matrix (H)
    kf.observation_matrices = np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0]]) 

    # Define the measurement noise covariance matrix (R)
    kf.observation_covariance = np.array([[standard_deviation_observation**2, 0],
                    [0, standard_deviation_observation**2]])
    
    
    
    return kf

### Applying Kalman Filter on a random selection of flight IDs, on the entire duration of the flight 

In [ ]:
# Randomly select flight IDs from your dataset
selected_flight_ids = random.sample(list(datas.keys()), 5)  # Change 5 to the desired number of flights

# Dictionary to store results for analysis
experiment_results = {}

# Loop through each selected flight ID
for flight_id in selected_flight_ids:
    print(f"Processing flight: {flight_id}")
    flight = deepcopy(datas[flight_id])  # Make a deep copy of the flight data to avoid altering the original 

    actual_data = flight.data

    # Ensure the flight exists and has data
    if flight is None or flight.data.empty:
        print(f"Skipping flight {flight_id} due to missing or empty data.")
        continue
            
    # Generate radar data for the same time range and ensure it exists
    radar_data = get_radar_data_for_flight(flight).data
    if radar_data.empty:
        print(f"Skipping flight {flight_id} due to lack of radar data.")
        continue

    # Construct and apply the Kalman filter based on radar data
    kf = construct_kalman_filter(radar_data) 
    radar_measurements = np.column_stack((radar_data.x, radar_data.y))  
    filtered_state_means, filtered_state_covariances = kf.filter(radar_measurements)
   
    # Apply smoothing to the Kalman filter
    smoothed_state_means, smoothed_state_covariances = kf.smooth(radar_measurements)

    # Only proceed if length of radar_data, fitlered_state_means and smoothed_state_means match; this is to ensure that each data set corresponds to the same number of time points
    # filtered_state_means is a 2D array that holds the estimated states and velocities for each time point after applying the Kalman filter. 
    # these state estimates are generated based on the noisy radar data and represents an estimate that aims to be closer to the true flight path 
    # smoothed_state_means contains an improved estimate of the flight's position by considering future and past observations
    # we need to apply the filtered estimates back into the context of the original flight's data structure --> we need to replace the original noisy radar positions with the filtered estimates 
    if len(radar_data) == len(filtered_state_means) and len(radar_data) == len(smoothed_state_means):
       # Create two separate deep copies for filtered and smoothed data updates
        filtered_flight_data = deepcopy(flight.data)
        smoothed_flight_data = deepcopy(flight.data)

        # Update 'x', 'y' in the deep copies based on filtered/smoothed values
        for idx, _ in enumerate(filtered_state_means):  # Use enumerate to get the index
            if idx in flight.data.index:
                # Update for filtered states
                filtered_flight_data.at[idx, 'x'] = filtered_state_means[idx, 0]
                filtered_flight_data.at[idx, 'y'] = filtered_state_means[idx, 1]
                
                # Update for smoothed states
                smoothed_flight_data.at[idx, 'x'] = smoothed_state_means[idx, 0]
                smoothed_flight_data.at[idx, 'y'] = smoothed_state_means[idx, 1]

        # Convert updated x, y back to lat, lon for both filtered and smoothed data
        filtered_flight = deepcopy(flight)
        filtered_flight.data = filtered_flight_data
        set_lat_lon_from_x_y(filtered_flight) # This is to convert the cartesian (x,y) coordiantes into lat and lon

        smoothed_flight = deepcopy(flight)
        smoothed_flight.data = smoothed_flight_data
        set_lat_lon_from_x_y(smoothed_flight)
        
        # Store the filtering results for further analysis
        experiment_results[flight_id] = {
            'actual_data': actual_data, 
            'radar_data': radar_data,
            'filtered_means': filtered_flight.data,
            'smoothed_means': smoothed_flight.data
    }
    

To view the contents of `experiment_results` for each flight_id: 

In [ ]:
for flight_id, results in experiment_results.items():
    print(f"Results for Flight ID: {flight_id}")
    print("Actual Data:")
    print(results['actual_data'])
    print("Radar Data:")
    print(results['radar_data'])
    print("Filtered Means:")
    print(results['filtered_means'])
    print("Smoothed Means:")
    print(results['smoothed_means'])
    print("\n")  


# Data Visualisation

Task 3: Plotting radar-simulated data against the original flight data to observe the difference 

In [ ]:
plot_actual_radar_longitude(actual_data, radar_data, 5, 5)

In [ ]:
plot_actual_radar_latitude(actual_data, radar_data, 5, 5)

In [ ]:
plot_actual_radar_all(actual_data, radar_data, 5, 5, 5)

### TASK 5: Plot filtered position estimates alongisde the original track 

In [ ]:
plot_actual_radar_latitude(actual_data, filtered_flight_data, 5, 5, radar_data_label_latitude="Latitude (Filtered)")

In [ ]:
plot_actual_radar_longitude(actual_data, filtered_flight_data, 5, 5, radar_data_label_longitude="Longitude (Filtered)")

In [ ]:
plot_actual_radar_all(actual_data, filtered_flight_data, 5, 5, radar_data_label_longitude= "Longitude (Filtered)", radar_data_label_latitude="Latitude (Filtered)")

# Error Measurement and Model Evaluation

### Define Function for calculating mean and maximum distance

In [ ]:
def calculate_mean_max_distance(data_1, data_2):
    """returns (mean distance, max distance) of two geo coordinated data.  

    Args:
        data_1 (pandas.DataFrame): DataFrame of tuples with 'latitude' and 'longitude' values
        data_2 (pandas.DataFrame): DataFrame of tuples with 'latitude' and 'longitude' values
    """
    # Initialize list to store distances
    distances = []

    # Iterate through the filtered positions and the original data
    for (data_1_lat, data_1_lon), (data_2_lat, data_2_lon) in zip(zip(data_1.latitude, data_1.longitude), zip(data_2.latitude, data_2.longitude)):
        if pd.isna(data_1_lat) or pd.isna(data_1_lon) or pd.isna(data_2_lat) or pd.isna(data_2_lon):
            continue  # Skip this pair if any NaN values are found
        # Calculate distance between filtered and original positions
        distance = dist.geodesic((data_1_lat, data_1_lon), (data_2_lat, data_2_lon)).meters  # geodesic returns the distance in meters
        distances.append(distance)

    # Compute the mean and maximal distances
    mean_distance = np.mean(distances)
    max_distance = np.max(distances)
    
    return mean_distance, max_distance

### TASK 6: Calculating the mean and max distance

In [205]:
for flight_id, results in experiment_results.items():
    actual_data = results['actual_data']
    filtered_data = results['filtered_means'] 

    mean_distance, max_distance = calculate_mean_max_distance(actual_data, filtered_data)
    
    print(f"Flight ID: {flight_id}")
    print(f"Mean Distance: {mean_distance} meters")
    print(f"Max Distance: {max_distance} meters")
    

Flight ID: REGA1_018
Mean Distance: 6102.605936323984 meters
Max Distance: 10248.618368998072 meters
Flight ID: VHOMS_054
Mean Distance: 856.2746710901737 meters
Max Distance: 6464.011350103178 meters
Flight ID: FAF4011_009
Mean Distance: 275984.8301197562 meters
Max Distance: 446350.6178366229 meters
Flight ID: CALIBRA_026
Mean Distance: 11029.914060992547 meters
Max Distance: 29109.970302704744 meters
Flight ID: VOR05_038
Mean Distance: 27018.777993784268 meters
Max Distance: 77688.65335830649 meters


Use smoothing to compute estimates for all states based on all the available data. How much
better are the smoothed tracks, compared to the filtered ones? Are there specific areas in
the flights where smoothed tracks are better?

To evaluate how much better the smoothed tracks are compared to the filtered ones, we need to compare the average and maximum distances from the actual flight path for both the filtered and smoothed estimates.    apply this function separately for the filtered and smoothed data against the actual data, then compare these values.



In [206]:
# we want to compare the accuracy of flight positions with actual flight data vs. filtered and smooth flight paths
# Initialize lists to store mean and max distances for filtered and smoothed data
filtered_distances = {'mean': [], 'max': []}
smoothed_distances = {'mean': [], 'max': []}

# Loop through each selected flight ID and calculate distances
for flight_id, results in experiment_results.items():
    actual_data = results['actual_data']
    filtered_data = results['filtered_means']
    smoothed_data = results['smoothed_means']

    # Calculate distances for filtered and smoothed data
    mean_filtered_distance, max_filtered_distance = calculate_mean_max_distance(actual_data, filtered_data)
    mean_smoothed_distance, max_smoothed_distance = calculate_mean_max_distance(actual_data, smoothed_data)

    # Store the distances
    filtered_distances['mean'].append(mean_filtered_distance)
    filtered_distances['max'].append(max_filtered_distance)
    smoothed_distances['mean'].append(mean_smoothed_distance)
    smoothed_distances['max'].append(max_smoothed_distance)

    # Print out distances for the current flight
    print(f"Flight ID: {flight_id}")
    print(f"Mean Filtered Distance: {mean_filtered_distance:.2f} meters")
    print(f"Max Filtered Distance: {max_filtered_distance:.2f} meters")
    print(f"Mean Smoothed Distance: {mean_smoothed_distance:.2f} meters")
    print(f"Max Smoothed Distance: {max_smoothed_distance:.2f} meters")
    print("-" * 50)  # Print a separator for readability

# After the loop, compute the averages across all flights
average_filtered_mean_distance = np.mean(filtered_distances['mean'])
average_filtered_max_distance = np.mean(filtered_distances['max'])
average_smoothed_mean_distance = np.mean(smoothed_distances['mean'])
average_smoothed_max_distance = np.mean(smoothed_distances['max'])

# Print out the overall averages
print("Overall Averages:")
print(f"Average Mean Distance for Filtered Data: {average_filtered_mean_distance:.2f} meters")
print(f"Average Max Distance for Filtered Data: {average_filtered_max_distance:.2f} meters")
print(f"Average Mean Distance for Smoothed Data: {average_smoothed_mean_distance:.2f} meters")
print(f"Average Max Distance for Smoothed Data: {average_smoothed_max_distance:.2f} meters")


Flight ID: REGA1_018
Mean Filtered Distance: 6102.61 meters
Max Filtered Distance: 10248.62 meters
Mean Smoothed Distance: 6102.34 meters
Max Smoothed Distance: 10217.51 meters
--------------------------------------------------
Flight ID: VHOMS_054
Mean Filtered Distance: 856.27 meters
Max Filtered Distance: 6464.01 meters
Mean Smoothed Distance: 802.78 meters
Max Smoothed Distance: 6156.90 meters
--------------------------------------------------
Flight ID: FAF4011_009
Mean Filtered Distance: 275984.83 meters
Max Filtered Distance: 446350.62 meters
Mean Smoothed Distance: 275981.82 meters
Max Smoothed Distance: 446323.34 meters
--------------------------------------------------
Flight ID: CALIBRA_026
Mean Filtered Distance: 11029.91 meters
Max Filtered Distance: 29109.97 meters
Mean Smoothed Distance: 11029.19 meters
Max Smoothed Distance: 29027.82 meters
--------------------------------------------------
Flight ID: VOR05_038
Mean Filtered Distance: 27018.78 meters
Max Filtered Distan

In [ ]:
# Plotting the comparison for one selected flight
plt.figure(figsize=[15, 10])
plt.plot(actual_data['longitude'], actual_data['latitude'], 'g-', label='Actual Path')
plt.scatter(radar_data['longitude'], radar_data['latitude'], c='r', marker='o', label='Radar Measurements')
plt.plot(filtered_state_means[:, 0], filtered_state_means[:, 1], 'b-', label='Filtered Path')
plt.plot(smoothed_state_means[:, 0], smoothed_state_means[:, 1], 'c-', label='Smoothed Path')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.title(f"Flight Path Comparison for {flight_id}")
plt.show()
